# Chapter 9: Analysis of Chemical Reactions

## 9.1. Calculation of Reaction Thermodynamics

In the following section, we will calculate the changes in enthalpy, entropy and Gibbs free energy of Diels-Alder reactions:

### 9.1.1. Calculation of Thermodynamic State Functions of Reactants and Products

In [ ]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem
from utils import View3DModel
import psi4
import py3Dmol

In [ ]:
# Create a molecule of a diene (buta-1,3-diene)
diene_mol = Chem.MolFromSmiles('C=CC=C')

# Prepare the molecule
diene_mol = Chem.AddHs(diene_mol)
AllChem.EmbedMolecule(diene_mol, useRandomCoords=True)
Chem.rdMolTransforms.SetDihedralDeg(diene_mol.GetConformer(), 0, 1, 2, 3, 0) # Set the C-C-C-C dihedral angle to 0 (s-cis conformation)
AllChem.UFFOptimizeMolecule(diene_mol, maxIters=200)

# View 3D model with py3Dmol
View3DModel(diene_mol)

In [ ]:
# Create a molecule of a dienophile (ethylene)
dienophile_mol = Chem.MolFromSmiles('C=C')

# Prepare the molecule
dienophile_mol = Chem.AddHs(dienophile_mol)
AllChem.EmbedMolecule(dienophile_mol, useRandomCoords=True)
AllChem.UFFOptimizeMolecule(dienophile_mol, maxIters=200)

# View 3D model with py3Dmol
View3DModel(dienophile_mol)

In [ ]:
# Create a molecule of the product
product_mol = Chem.MolFromSmiles('C1CCC=CC1')

# Prepare the molecule
product_mol = Chem.AddHs(product_mol)
AllChem.EmbedMolecule(product_mol, useRandomCoords=True)
AllChem.UFFOptimizeMolecule(product_mol, maxIters=200)

# View 3D model with py3Dmol
View3DModel(product_mol)

In [ ]:
# Set the number of threads and memory limit
psi4.set_num_threads(16)
psi4.set_memory(16*1024*1024*1024) # 16 GB

In [ ]:
# Set calculation options
psi4.set_options({
    'basis': '6-31G*',
    'scf_type': 'DF',
    'reference': 'RHF'
})

In [ ]:
# Define a function that calculate the thermodynamic properties of molecule
def calculate_thermo_properties(mol, temperature, pressure):
    # Write the geometry to XYZ string
    xyz_string = Chem.MolToXYZBlock(mol)

    # Get the psi4 geometry
    geometry = psi4.geometry(xyz_string)
        
    # Set the temperature and pressure
    psi4.set_options({
        'T': temperature,
        'P': pressure * 101325
    })
    
    # Perform geometry optimization
    psi4.optimize('b3lyp', molecule=geometry)

    # Perform frequency analysis
    psi4.frequencies('b3lyp', molecule=geometry)
    
    # Extract thermodynamic parameters
    h = psi4.core.variable('ENTHALPY') * psi4.constants.hartree2kcalmol # Equal to E_h + h_corr
    g = psi4.core.variable('GIBBS FREE ENERGY') * psi4.constants.hartree2kcalmol # Equal to E_h + g_corr
    s = (h - g) / temperature
    return h, s, g

In [ ]:
# Set the temperature and pressure
temperature = 298.15 # K
pressure = 1 # atm

In [ ]:
# Calculate thermodynamic properties of the diene
diene_h, diene_s, diene_g = calculate_thermo_properties(diene_mol, temperature, pressure)
print([diene_h, diene_s, diene_g])

In [ ]:
# Calculate thermodynamic properties of the dienophile
dienophile_h, dienophile_s, dienophile_g = calculate_thermo_properties(dienophile_mol, temperature, pressure)
print([dienophile_h, dienophile_s, dienophile_g])

In [ ]:
# Calculate thermodynamic properties of the product
product_h, product_s, product_g = calculate_thermo_properties(product_mol, temperature, pressure)
print([product_h, product_s, product_g])

### 9.1.2. Calculation of Thermodynamic State Functions of the Reaction

Using Hess's law, we can calculate $\Delta{H}$, $\Delta{S}$, and $\Delta{G}$ of the reaction:

In [ ]:
# Calculate the changes in enthalpy, entropy and Gibbs free energy for the reaction
dh = product_h - (diene_h + dienophile_h)
ds = product_s - (diene_s + dienophile_s)
dg = product_g - (diene_g + dienophile_g)

print(f'ΔH = {dh:.4f} kcal/mol')
print(f'ΔS = {ds:.4f} kcal/mol')
print(f'ΔG = {dg:.4f} kcal/mol')